# **Required Package**

In [1]:
!pip install langchain-google-genai PyPDF2

# **Import package**

In [2]:
import PyPDF2
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import os
import re
from typing import List, Dict, Tuple

# **Assistant Function**

In [16]:
class StudyAssistant:
    def __init__(self, gemini_api_key: str):
        os.environ["GOOGLE_API_KEY"] = gemini_api_key


        self.llm = ChatGoogleGenerativeAI(
            model="models/gemini-1.5-flash",
            temperature=0.7,
            max_output_tokens=2048
        )

        self.summary_template = PromptTemplate(
            input_variables=["content"],
            template="""
            Summarize the following study material into concise bullet points.
            Focus on key concepts, definitions, and main ideas.

            Study Material:
            {content}

            Summary:
            """
        )

        self.quiz_template = PromptTemplate(
            input_variables=["summary"],
            template="""
            Generate 5 multiple-choice questions based on this summary.
            Format each question exactly as shown:

            Question: [Question text]
            a) [Option A]
            b) [Option B]
            c) [Option C]
            d) [Option D]
            Correct Answer: [a/b/c/d]

            Summary:
            {summary}

            Quiz Questions:
            """
        )

        self.summary_chain = LLMChain(llm=self.llm, prompt=self.summary_template)
        self.quiz_chain = LLMChain(llm=self.llm, prompt=self.quiz_template)

    def extract_pdf_content(self, pdf_path: str) -> str:
        text = ""
        with open(pdf_path, "rb") as file:
            reader = PyPDF2.PdfReader(file)
            for page in reader.pages:
                if page_text := page.extract_text():
                    text += page_text
        return text.strip()

    def generate_summary(self, content: str) -> str:
        if len(content) > 6000:
            content = content[:6000] + "..."
        return self.summary_chain.run(content=content).strip()

    def generate_quiz(self, summary: str) -> List[Dict]:
        quiz_text = self.quiz_chain.run(summary=summary)
        return self.parse_quiz_questions(quiz_text)

    def parse_quiz_questions(self, quiz_text: str) -> List[Dict]:
        questions = []
        lines = quiz_text.split('\n')

        current_question = None
        current_options = {}
        current_answer = None

        for line in lines:
            line = line.strip()
            if not line:
                continue

            if line.startswith('Question:'):
                if current_question and len(current_options) >= 4 and current_answer:
                    questions.append({
                        'question': current_question,
                        'options': current_options.copy(),
                        'correct_answer': current_answer
                    })
                current_question = line.replace('Question:', '').strip()
                current_options = {}
                current_answer = None

            elif re.match(r'^[a-d]\)', line):
                letter = line[0].lower()
                text = line[2:].strip()
                current_options[letter] = text

            elif 'Correct Answer:' in line:
                match = re.search(r'([a-d])', line.lower())
                if match:
                    current_answer = match.group(1)

        if current_question and len(current_options) >= 4 and current_answer:
            questions.append({
                'question': current_question,
                'options': current_options.copy(),
                'correct_answer': current_answer
            })

        return questions

    def process_pdf(self, pdf_path: str) -> Tuple[str, List[Dict]]:
        print(f"Processing PDF: {pdf_path}")
        content = self.extract_pdf_content(pdf_path)
        print(f"✓ Extracted {len(content)} characters from PDF")

        print("Generating summary...")
        summary = self.generate_summary(content)
        print("✓ Summary created")

        print("Generating quiz questions...")
        questions = self.generate_quiz(summary)
        print(f"✓ Generated {len(questions)} questions")

        return summary, questions

    def take_quiz(self, questions: List[Dict]):
        if not questions:
            print("No questions available!")
            return

        print("\n" + "="*50)
        print("           QUIZ TIME!")
        print("="*50)

        score = 0
        for i, q in enumerate(questions, 1):
            print(f"\nQuestion {i}: {q['question']}")
            for letter in ['a', 'b', 'c', 'd']:
                if letter in q['options']:
                    print(f"  {letter}) {q['options'][letter]}")

            while True:
                answer = input("Your answer (a/b/c/d): ").lower().strip()
                if answer in ['a', 'b', 'c', 'd']:
                    break
                print("Invalid input. Enter a/b/c/d.")

            if answer == q['correct_answer']:
                print("✓ Correct!")
                score += 1
            else:
                print(f"✗ Wrong! Correct answer: {q['correct_answer']}")

            print("-" * 30)

        percent = (score / len(questions)) * 100
        print(f"\nFinal Score: {score}/{len(questions)} ({percent:.1f}%)")

        if percent >= 80:
            print("🎉 Excellent!")
        elif percent >= 60:
            print("👍 Good job!")
        else:
            print("📚 Keep studying!")

# **Main Function**

In [21]:
import getpass

GEMINI_API_KEY = getpass.getpass("Enter your Gemini API key: ")

assistant = StudyAssistant(GEMINI_API_KEY)


pdf_path = "/content/Agentic_AI.pdf"
summary, questions = assistant.process_pdf(pdf_path)


print("\nSUMMARY:\n", summary)
print("\nQUIZ QUESTIONS:\n")
for i, q in enumerate(questions, 1):
    print(f"{i}. {q['question']}")
    for letter, option in q['options'].items():
        print(f"   {letter}) {option}")
    print(f"   Answer: {q['correct_answer']}")


assistant.take_quiz(questions)


Enter your Gemini API key: ··········
Processing PDF: /content/Agentic_AI.pdf
✓ Extracted 45804 characters from PDF
Generating summary...
✓ Summary created
Generating quiz questions...
✓ Generated 5 questions

SUMMARY:
 * **Agentic AI Defined:**  Autonomous AI systems that perceive, reason, and act to achieve goals, extending human capabilities.  Definitions vary across organizations (OpenAI, IBM, Google DeepMind, Artiﬁciality) but all emphasize these core components.

* **Historical Context:**  Evolved from early systems like Deep Blue to modern platforms like AutoGPT and LangChain, demonstrating rapid progress.

* **Core Components:**
    * **Perception:**  Environmental understanding via sensors and machine learning.
    * **Reasoning:**  Logical reasoning, probabilistic inference, and heuristic decision-making.
    * **Action:**  Task execution through pre-defined behaviors, learned skills, and adaptive control.

* **AI Personas:** Different combinations of perception, reasoning, a